# <h>WORKSHOP 09</h>

<b>Function to create a roof depending on a basic polyhedron (2D).
First of all, we need:
    <l><ul>-a set of vertices</ul><ul>-an angle</ul><ul>-a roof height</ul></l>
<br>
Libraries:
<l><ul>-larlib</ul><ul>-numpy</ul></b>

In [6]:
from larlib import *
from numpy import *

v = [[0.0,0.0],[0.0,2.0],[2.0,2.0],[2.0,0.0]]
alpha = PI/8
height = 2.0

This is a function to calculate the minimum and the maximum of a set of vertices on x and y axis.

In [2]:
def calculateMinMaxValues(points):
    """
    Function to calculate the minimum and the maximum of a set of vertices on x and y axis.
    
    Args:
    points: a set of vertices
    
    Returns:
    4 values corresponding to: xMin,yMin,xMax,yMax (wrapped in a set)
    """
    x = []
    y = []
    for p in points:
        x.append(p[0])
        y.append(p[1])
    xMax = np.amax(x)
    yMax = np.amax(y)
    xMin = np.amin(x)
    yMin = np.amin(y)
    
    return [xMax,yMax,xMin,yMin]

Function to calculate the higher internal points of the roof.

In [3]:
def calculateTopPoints(points,height,angle):
    """
    Function to calculate the higher internal points of the roof.
    
    Args:
    points: a set of vertices
    height: roof height
    angle: roof pitch
    
    Returns:
    A set containing the higher internal points of the roof.
    """    
    distance = math.atan2(height,1/math.sin(angle))
    xMax,yMax,xMin,yMin = calculateMinMaxValues(points)
    
    topPoints = []
    for i in range(0,len(points)):
        x = 0
        y = 0
        if points[i][0] == xMax:
            x = points[i][0]-distance
        elif points[i][0] == xMin or (points[i-1][0] <= points[i][0] and  points[i+1][0] >= points[i][0]):
            x = points[i][0]+distance
        
        if points[i][1] == yMin:
            y = points[i][1]+distance
        elif points[i][1] == yMax or (points[i-1][1] <= points[i][1] and points[i+1][1] >= points[i][0]):
            y = points[i][1]-distance
        point = [x,y]
        topPoints.extend([point])
        
    return topPoints

Function to create the roof (ceiling+roof).

In [4]:
def buildRoof(points,angle,h):
    """
    Function to create the roof (ceiling+roof).
    
    Args:
    points: a set of vertices
    h: roof height
    angle: roof pitch
    
    Returns:
    3D value of type HPC representing the roof.    
    """        
    topPoints = calculateTopPoints(points,h,angle)
        
    
    ceiling = STRUCT(AA(POLYLINE)([topPoints]))
    ceiling = SOLIDIFY(ceiling)
    ceiling = STRUCT([T(3)(h),ceiling])

    for p in points:
        p.append(0)
    for hp in topPoints:
        hp.append(h)
    newPoints = points+topPoints
    
    cells = []
    for j in range(1,len(points)+1):
        if j != len(points):
            cells.extend([[j,j+1,j+len(points),j+len(points)+1]])
        else:
            cells.extend([[j,1,j+len(points),j+1]])

    #Creazione del modello
    roof = MKPOL([newPoints,cells,None])
    roof = STRUCT([roof,ceiling])
    roof = OFFSET([.1,.1,.1])(roof)
    roof = COLOR(RED)(roof)

    return roof

In [7]:
roof = buildRoof(v,alpha,height)
VIEW(roof)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000B8323F0> >

Some examples:

![alt text](LRoof.png "Title")
![alt text](squareRoof.png "Title")